# Rwanda SAE Project: DHS & Census Harmonization

**Group**: 11  
**Date**: 2025  
**Project**: Small Area Estimation (SAE) with Elevation-Stunting Relationship

## Project Overview
This notebook demonstrates the integration of multiple data sources to analyze the relationship between **elevation** and **child stunting** in Rwanda, implementing **Small Area Estimation (SAE)** models for poverty mapping and nutritional analysis.

## Key Objectives
1.  **Harmonize Identifiers**: Create consistent 'Sector Codes' (`Code_Sect`) in both Census and DHS datasets to enable merging.
2.  **Integrate Data**: Merge Census data (providing Elevation/Altitude) with DHS data (providing Stunting Rates) at the Sector level.
3.  **Analyze Relationship**: Visualize the correlation between elevation and stunting using specific categories.
4.  **SAE Preparation & Mapping**: Create Poverty/Stunting Maps using Shapefiles.

## Data Sources
### 1. DHS Data (Demographic and Health Surveys)
- **Children's Recode** (`RWKR81FL (2).DTA`): Source of Stunting (Height-for-Age) data.
- **Household / Member Recode**: (`RWHR81FL (2).DTA`, `RWIR81FL (2).DTA`)

### 2. Census Data (Rwanda Population and Housing Census - PHC5)
- **Microdata** (`Census_data.sav`): Source of Elevation (`altitude`), geolocation, and population weights.

### 3. Spatial Data
- **Shapefile**: `Sector_Boundary_2012.shp`


In [ ]:
# Install Required Packages (run this cell first if packages are missing)
import sys
!{sys.executable} -m pip install -q geopandas pyreadstat plotly statsmodels

In [ ]:
# Step 1: Setup and Configuration
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
import pyreadstat
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

warnings.filterwarnings('ignore')

# Set up paths
base_path = Path(r"C:\Users\NISR\Desktop\gaga\AIMS\my project SAE")
output_path = base_path / "SEA RWANDA" / "Outputs"
output_path.mkdir(parents=True, exist_ok=True)

# DATA PATHS (As provided by User)
PATHS = {
    'dhs_kr': r"C:\Users\NISR\Desktop\USA World_Bank\My new Project\RWKR81FL (2).DTA",  # Children (Stunting)
    'dhs_hr': r"C:\Users\NISR\Desktop\USA World_Bank\My new Project\RWHR81FL (2).DTA",  # Household
    'dhs_ir': r"C:\Users\NISR\Desktop\USA World_Bank\My new Project\RWIR81FL (2).DTA",  # Individual
    'census': r"C:\Users\NISR\Desktop\USA World_Bank\Census_data.sav",                 # Census Data
    'shapefile': r"C:\Users\NISR\Desktop\gaga\AIMS\my project SAE\SEA RWANDA\Team_11_Final_Submission\Boundaries\shp\Sector_Boundary_2012\Sector_Boundary_2012.shp"
}

print("Setup completed successfully!")
print(f"Output directory: {output_path}")
print("Data paths defined:")
for k, v in PATHS.items():
    print(f"  {k}: {v}")

In [ ]:
# Step 2: DHS Processing - Stunting & Sector Aggregation

def process_dhs_children():
    """
    Process DHS Children's data to calculate stunting rates per Sector.
    Uses 'Code_Sect' if available, otherwise we might need to construct it.
    """
    print("\n--- Processing DHS Children (KR) Data ---")
    fpath = PATHS['dhs_kr']
    
    if not os.path.exists(fpath):
        print(f"[-] File not found: {fpath}"); return None
    
    try:
        # Load data (Stata format)
        df = pd.read_stata(fpath, convert_categoricals=False)
        print(f"[OK] Loaded {len(df)} records.")
        
        # Verify Code_Sect existence
        if 'Code_Sect' not in df.columns:
            print("[-] 'Code_Sect' NOT found. Checking alternatives...")
            aliases = ['shsect', 'ssect', 'sect']
            for a in aliases:
                if a in df.columns:
                    print(f"[+] Found alias '{a}', renaming to 'Code_Sect'")
                    df.rename(columns={a: 'Code_Sect'}, inplace=True)
                    break
        else:
            print("[+] 'Code_Sect' found.")
            
        # Calculate Stunting Indicators (WHO Standards)
        # hw70 = Height-for-age standard deviation (new WHO)
        if 'hw70' in df.columns:
            # Filter valid measurements (< 9990)
            mask_valid = df['hw70'] < 9990 
            
            # Stunted: < -2 SD (-200)
            df.loc[mask_valid, 'is_stunted'] = (df.loc[mask_valid, 'hw70'] < -200).astype(int)
            
            valid_count = mask_valid.sum()
            print(f"[OK] Stunting indicators calculated for {valid_count} children.")
        else:
            print("[-] 'hw70' (Height-for-Age) not found. Cannot calculate stunting.")
            return None

        # Aggregate by Sector
        if 'Code_Sect' in df.columns:
            sector_agg = df[mask_valid].groupby('Code_Sect').agg(
                stunting_rate=('is_stunted', 'mean'),
                child_count=('is_stunted', 'count'),
                mean_haz=('hw70', 'mean')
            ).reset_index()
            
            # Convert mean_haz back to normal units
            sector_agg['mean_haz'] = sector_agg['mean_haz'] / 100.0
            
            print(f"[OK] Aggregated to {len(sector_agg)} Sectors.")
            return sector_agg
        else:
            print("[-] Cannot aggregate: Sector Code missing.")
            return None

    except Exception as e:
        print(f"[-] Error processing DHS: {e}")
        return None

In [ ]:
# Step 3: Census Processing - ID Creation & Elevation Extraction

def process_census_data():
    """
    Process Census data to create 'Code_Sect' matching DHS,
    and extract Elevation (altitude).
    Uses SPSS Logic:
        District_code = (ml01 * 10) + ml02
        Sector_code = (District_code * 100) + ml03
    """
    print("\n--- Processing Census Data ---")
    fpath = PATHS['census']
    
    if not os.path.exists(fpath):
        print(f"[-] File not found: {fpath}"); return None

    try:
        # LOAD OPTIMIZATION: Only load necessary columns to avoid MemoryError
        target_cols = ['ml01', 'ml02', 'ml03', 'altitude', 'latitude', 'longitude']
        print(f"Loading Census data (subset: {target_cols})...")
        
        # 1. Read Metadata to check column names (case sensitivity)
        _, meta = pyreadstat.read_sav(fpath, metadataonly=True)
        all_cols = meta.column_names
        
        # 2. Match target columns insensitive to case
        actual_cols = []
        for target in target_cols:
            for col in all_cols:
                if target.lower() == col.lower():
                    actual_cols.append(col)
                    break
        
        # 3. Read Data using usecols
        df, meta = pyreadstat.read_sav(fpath, usecols=actual_cols)
        print(f"[OK] Loaded {len(df)} Census records.")
        
        # Normalize column names to lowercase
        df.columns = [c.lower() for c in df.columns]
        
        # --- IMPLEMENT SPSS LOGIC FOR ID CREATION ---
        print("Creating Identifier Variables...")
        
        # Ensure numeric types
        for c in ['ml01', 'ml02', 'ml03']:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)

        # 1. District Code: (Province * 10) + District
        df['District_code'] = (df['ml01'] * 10) + df['ml02']
        
        # 2. Sector Code: (District * 100) + Sector
        df['Code_Sect'] = (df['District_code'] * 100) + df['ml03']
        
        print(f"[+] Created 'Code_Sect'. Sample: {df['Code_Sect'].head().tolist()}")

        # Aggregate Elevation by Sector
        print("Aggregating Elevation by Sector...")
        sector_summary = df.groupby('Code_Sect').agg(
            elevation=('altitude', 'mean'),
            latitude=('latitude', 'mean'),
            longitude=('longitude', 'mean'),
            census_pop=('ml01', 'count')
        ).reset_index()
        
        print(f"[OK] Processed {len(sector_summary)} Census Sectors.")
        return sector_summary

    except Exception as e:
        print(f"[-] Error processing Census: {e}")
        return None

In [ ]:
# Step 4: Data Integration & Feature Engineering

def create_integrated_dataset(dhs_data, census_data):
    print("\n--- Merging Datasets & creating Features ---")
    
    if dhs_data is None or census_data is None:
        print("[-] One or both datasets missing. Aborting merge.")
        return None
        
    # Standardize Merge Key
    dhs_data['Code_Sect'] = dhs_data['Code_Sect'].astype(int)
    census_data['Code_Sect'] = census_data['Code_Sect'].astype(int)
    
    # Merge
    merged = pd.merge(dhs_data, census_data, on='Code_Sect', how='inner')
    print(f"[OK] Merged Data Shape: {merged.shape}")
    print(f"[OK] Matched Sectors: {merged['Code_Sect'].nunique()}")
    
    # Create Elevation Categories for Analysis
    # 1: Lowland, 2: Midland, 3: Highland, 4: Mountain
    conditions = [
        (merged['elevation'] < 1500),
        (merged['elevation'] >= 1500) & (merged['elevation'] < 1800),
        (merged['elevation'] >= 1800) & (merged['elevation'] < 2200),
        (merged['elevation'] >= 2200)
    ]
    choices = [1, 2, 3, 4]
    
    merged['elevation_cat'] = np.select(conditions, choices, default=1)
    
    # Create 'stunting_level' for visualization coloring (e.g., quantiles)
    merged['stunting_level'] = pd.qcut(merged['stunting_rate'], q=4, labels=False) + 1
    
    # Save
    save_path = output_path / "rwanda_sae_integrated_final.csv"
    merged.to_csv(save_path, index=False)
    print(f"[+] Saved integrated dataset to: {save_path}")
    return merged

In [ ]:
# Step 5: Elevation-Stunting Relationship Analysis

def analyze_elevation_stunting_relationship(df_sectors):
    """Analyze the relationship between elevation and stunting"""
    
    print("\n--- Analyzing Elevation-Stunting Relationship ---")
    
    # Set up plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    # Basic statistics
    print(f"Number of sectors: {len(df_sectors)}")
    print(f"Stunting rate - Mean: {df_sectors['stunting_rate'].mean():.3f}")
    print(f"Elevation - Mean: {df_sectors['elevation'].mean():.1f}m")
    
    # Correlation analysis
    correlation = df_sectors['stunting_rate'].corr(df_sectors['elevation'])
    print(f"Correlation between elevation and stunting: {correlation:.3f}")
    
    # Scatter plot with regression line
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(df_sectors['elevation'], df_sectors['stunting_rate'], 
                         c=df_sectors['stunting_level'], cmap='RdYlGn_r', 
                         s=100, alpha=0.7)
    
    z = np.polyfit(df_sectors['elevation'], df_sectors['stunting_rate'], 1)
    p = np.poly1d(z)
    plt.plot(df_sectors['elevation'], p(df_sectors['elevation']), "r--", alpha=0.8)
    
    plt.xlabel('Elevation (meters)')
    plt.ylabel('Stunting Prevalence')
    plt.title('Stunting Prevalence vs Elevation in Rwanda Sectors')
    plt.colorbar(scatter, label='Stunting Level (Quartile)')
    plt.grid(True, alpha=0.3)
    plt.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
             transform=plt.gca().transAxes, bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.8))
    plt.savefig(output_path / "elevation_stunting_scatter.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Box plot by elevation category
    plt.figure(figsize=(10, 6))
    elevation_cat_names = ['Lowland', 'Midland', 'Highland', 'Mountain']
    box_data = [df_sectors[df_sectors['elevation_cat'] == cat]['stunting_rate'].values for cat in [1, 2, 3, 4]]
    box_plot = plt.boxplot(box_data, labels=elevation_cat_names, patch_artist=True)
    colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightsalmon']
    for patch, color in zip(box_plot['boxes'], colors):
        patch.set_facecolor(color)
    plt.xlabel('Elevation Category')
    plt.ylabel('Stunting Prevalence')
    plt.title('Stunting Prevalence by Elevation Category')
    plt.grid(True, alpha=0.3)
    plt.savefig(output_path / "stunting_by_elevation_box.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Summary
    print("\nStunting by Elevation Category Summary:")
    elevation_summary = df_sectors.groupby('elevation_cat').agg({
        'stunting_rate': ['mean', 'std', 'count'],
        'elevation': 'mean'
    }).round(3)
    elevation_summary.index = elevation_cat_names[:len(elevation_summary)]
    print(elevation_summary)
    
    return correlation

In [ ]:
# Step 6: Poverty Mapping & Priority Zones

def create_poverty_stunting_map(df_sectors):
    """Load Shapefile and map stunting rates"""
    print("\n--- Creating Stunting & Poverty Maps ---")
    shp_path = PATHS['shapefile']
    
    if not os.path.exists(shp_path):
        print(f"[-] Shapefile not found: {shp_path}")
        return
        
    # Load Shapefile
    gdf = gpd.read_file(shp_path)
    print(f"[OK] Loaded Shapefile: {len(gdf)} records.")
    
    # Find matching column dynamically
    match_col = None
    for col in gdf.columns:
        if 'sect' in col.lower() and 'id' in col.lower():
            match_col = col
            break
    
    if match_col:
        print(f"[+] Found Sector ID column: {match_col}")
        gdf['Code_Sect'] = pd.to_numeric(gdf[match_col], errors='coerce').fillna(0).astype(int)
    elif 'IF_ID' in gdf.columns:
         gdf['Code_Sect'] = pd.to_numeric(gdf['IF_ID'], errors='coerce').fillna(0).astype(int)
    else:
        print("[-] Could not identify Sector ID in shapefile.")
        gdf['Code_Sect'] = pd.to_numeric(gdf.get('Sector_ID', 0), errors='coerce')

    # Merge
    map_data = gdf.merge(df_sectors, on='Code_Sect', how='inner')
    print(f"[OK] Merged Map Data: {len(map_data)} sectors.")
    
    if len(map_data) == 0:
        print("[-] No matching sectors found! Check IDs.")
        return

    # Plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    map_data.plot(column='stunting_rate', ax=ax, legend=True, 
                  cmap='RdYlGn_r', 
                  legend_kwds={'label': "Stunting Prevalence", 'orientation': "horizontal"})
    
    plt.title('Stunting Prevalence by Sector (Rwanda)', fontsize=15)
    plt.axis('off')
    
    map_path = output_path / "stunting_map_rwanda.png"
    plt.savefig(map_path, dpi=300, bbox_inches='tight')
    print(f"[+] Map saved to {map_path}")
    plt.show()

    # Priority Zones (Top 10% highest stunting)
    priority_threshold = df_sectors['stunting_rate'].quantile(0.90)
    print(f"\nPriority Zone Threshold (>90th percentile): {priority_threshold:.3f}")
    
    priority_sectors = df_sectors[df_sectors['stunting_rate'] > priority_threshold]
    print(f"Identified {len(priority_sectors)} Priority Sectors.")
    
    # Save Priority List
    prio_path = output_path / "priority_sectors_intervention.csv"
    priority_sectors.to_csv(prio_path, index=False)
    print(f"[+] Priority list saved to {prio_path}")

In [ ]:
# Execute Entire Workflow
dhs_sectors = process_dhs_children()
census_sectors = process_census_data()

final_df = create_integrated_dataset(dhs_sectors, census_sectors)

if final_df is not None and not final_df.empty:
    # Analysis
    analyze_elevation_stunting_relationship(final_df)
    
    # Mapping
    create_poverty_stunting_map(final_df)
    
    print("\n=== SAE Analysis Complete ===")